Library

In [1]:
import pandas as pd
import os
import re

In [2]:
API_KEY = "sk-ms7SU43E34tS9UJks5RD2KM3m1JumOR2pM73Dk95VzKjM6TZ"

API_KEY = API_KEY or os.getenv("H2O_GPT_E_API_KEY")

if not API_KEY:
    raise ValueError("Please configure h2ogpte API key")

REMOTE_ADDRESS = "https://h2ogpte.genai.h2o.ai"

from h2ogpte import H2OGPTE

client = H2OGPTE(address=REMOTE_ADDRESS, api_key=API_KEY)

Common issue extration by topic

In [3]:
tor = pd.read_csv('/Users/blabbyduck/Desktop/Y3S2/DSA3101/DSA3101-Project/csv_output/TopicsofReviews.csv')

In [4]:
def get_review_issue(topic_name):
    issue1 = tor.loc[tor['Topic_Name'] == topic_name]
    return issue1

In [5]:
#first convert the extracted/filtered topic reviews extracted into strings
def convert_str(topic_review):
    compiled_review = topic_review['review'].to_string()

    comp_rev = ""
    for row in compiled_review:
        comp_rev += row
    return(comp_rev) #returns a long string

In [7]:
#extract common issues and its corresponding review number, but all in one string
def common_issues(review):
    extract = client.extract_data(
        text_context_list= [review],
        #pre_prompt_extract="Pay attention and look at all people. Your job is to collect their names.\n",
        prompt_extract="Extract all the common issues faced by the application according to the reviews, as well as their corresponding index after the identified issues. Do not return positive. Strictly return in the following format, issues : indexes without additional information"
    )
    # List of LLM answers per text input
    sol = []
    for extract_list_item in extract.content:
        for s in extract_list_item.split("\n"):
            s = re.sub('\W+',' ', s)
            sol.append(s)
    return sol

<>:12: SyntaxWarning: invalid escape sequence '\W'
<>:12: SyntaxWarning: invalid escape sequence '\W'
/var/folders/1l/pycwvg157rv_sn0s68n1_y8c0000gn/T/ipykernel_9223/4238437813.py:12: SyntaxWarning: invalid escape sequence '\W'
  s = re.sub('\W+',' ', s)


In [8]:
#format it so that they are in dic form, for better df manipulation
def extract_review(issue_ind):
    result_dict = {}
    for item in issue_ind:
        # Use regex to extract the string part without numbers
        string = re.sub(r'\d+', '', item)  # Replace all digits with an empty string
        string = string.strip()  # Remove leading and trailing whitespaces
        
        numbers = [int(num) for num in item.split() if num.isdigit()]  # Extract all numbers in the string
        if string in result_dict:
            result_dict[string].extend(numbers)
        else:
            result_dict[string] = numbers
    
    return result_dict

In [33]:
#combine the reviews on issues tgt. Damn just combine everything above TT
def issue_df(topic):
    issue1 = tor.loc[tor['Topic_Name'] == topic]
    review_str = convert_str(issue1)
    review_lst = common_issues(review_str)
    dic = extract_review(review_lst)
    
    # Get all indices covered by the dictionary values
    covered_indices = set(index for indices in dic.values() for index in indices)

    # Get all indices in the tof dataset
    all_indices = set(issue1['index'])

    # Get the remaining indices
    remaining_indices = all_indices - covered_indices

    # Create an empty list to hold individual DataFrames
    dfs = []

    # Iterate over the dictionary items
    for key, indices in dic.items():
    # Create a DataFrame for each key-value pair
        indices_with_reviews = [index for index in indices if index in issue1.index]
        df = pd.DataFrame({'key': [key]*len(indices_with_reviews), 'review': [issue1.loc[i, 'review'] for i in indices_with_reviews]})
        dfs.append(df)

# Add remaining indices under "Others"
    if remaining_indices:
        others_indices_with_reviews = [index for index in remaining_indices if index in issue1.index]
        others_df = pd.DataFrame({'key': ['Others']*len(others_indices_with_reviews), 'review': [issue1.loc[i, 'review'] for i in others_indices_with_reviews]})
        dfs.append(others_df)

    # Concatenate all DataFrames together
    result_df = pd.concat(dfs, ignore_index=True)

    return(result_df)


To use the function, simply type in the topic which you want to extract the specific issues from

In [36]:
the_df = issue_df("Login & Account Setup")
the_df

,key,review
0,Difficulty logging in,Unable to login or use the forgot password fun...
1,Difficulty logging in,Wasn't able to log in since 7 Sept. Called the...
2,Difficulty logging in,Can't login now. Makes me feel like being SCAMMED
3,Difficulty logging in,The bank is not ready at all. I'm your ecosyst...
4,Unable to set up account,Cannot set up my account in redmi 8T. Reach th...
5,Unable to set up account,"After signed up and asked for my info,straight..."
6,Poor customer support,Buggy app and bad customer support
7,Poor customer support,"Disappointing app experience, awful faux frien..."
8,Buggy app,Buggy app and bad customer support
9,Buggy app,Cant launch. Stalled at invisible layer. Then ...


In [37]:
#save df
the_df.to_csv('/Users/blabbyduck/Desktop/Y3S2/DSA3101/DSA3101-Project/Login & Account Setup.csv')

Debug, ignore below

In [31]:
#debug
issue1 = tor.loc[tor['Topic_Name'] == "User Interface"]
review_str = convert_str(issue1)
review_lst = common_issues(review_str)
review_lst

[' Poor login functionality 37 169 122 196 294',
 ' Fingerprint login not working 108',
 ' Unclear error messages 233',
 ' Terrible user experience 238 313',
 ' Lack of transparency 205',
 ' Issues with interest rates 36 146',
 ' Difficulty in signing up 254',
 ' Compatibility issues 276 310',
 ' Slow response time 283',
 ' Occasional freezing 233',
 ' Not suitable for non residents 254',
 ' No response after clicking login 299']

In [32]:
#to get dictionary
dic = extract_review(review_lst)
dic

{'Poor login functionality': [37, 169, 122, 196, 294],
 'Fingerprint login not working': [108],
 'Unclear error messages': [233],
 'Terrible user experience': [238, 313],
 'Lack of transparency': [205],
 'Issues with interest rates': [36, 146],
 'Difficulty in signing up': [254],
 'Compatibility issues': [276, 310],
 'Slow response time': [283],
 'Occasional freezing': [233],
 'Not suitable for non residents': [254],
 'No response after clicking login': [299]}